<a href="https://colab.research.google.com/github/omkarpat/eeg-mood-classification/blob/master/Baseline_V2_ModelTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 240/Project/Data/spectrogram_images/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive/CSE 240/Project/Data/spectrogram_images/'
/content/gdrive/My Drive/CSE 240/Project/Data/spectrogram_images


In [18]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold

%tensorflow_version 2.x
import tensorflow
print(tensorflow.__version__)
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate, SeparableConv2D, MaxPooling3D, Conv3D, DepthwiseConv2D
from tensorflow.keras.models import model_from_json

2.1.0


In [19]:
base_path = "."
concentration_data_array = None
relaxed_data_array = None
print("loading data from .npy files...")
for filename in os.listdir(base_path):
  if "concentration_array_updated_spectrograms" in filename and "final" not in filename:
    print(filename)
    concentration_data_array = np.concatenate((concentration_data_array, np.load(filename))) if concentration_data_array is not None else np.load(filename)
  if "relaxed_array_updated_spectrograms" in filename:
    print(filename)
    relaxed_data_array = np.concatenate((relaxed_data_array, np.load(filename))) if relaxed_data_array is not None else np.load(filename)
print(concentration_data_array.shape, relaxed_data_array.shape)
print(len(concentration_data_array), len(relaxed_data_array))

loading data from .npy files...
concentration_array_updated_spectrograms_1.npy
concentration_array_updated_spectrograms_2.npy
concentration_array_updated_spectrograms_3.npy
concentration_array_updated_spectrograms_4.npy
concentration_array_updated_spectrograms_5.npy
concentration_array_updated_spectrograms_6.npy
relaxed_array_updated_spectrograms_1.npy
relaxed_array_updated_spectrograms_2.npy
relaxed_array_updated_spectrograms_3.npy
relaxed_array_updated_spectrograms_4.npy
relaxed_array_updated_spectrograms_5.npy
relaxed_array_updated_spectrograms_final.npy
(2160, 29, 43, 42) (2124, 29, 43, 42)
2160 2124


In [20]:
X = np.concatenate((relaxed_data_array, concentration_data_array))
Y = np.concatenate((np.zeros((len(relaxed_data_array),1)),np.ones((len(concentration_data_array),1))))
print(X.shape, Y.shape)
relaxed_data_array, concentration_data_array = None, None

(4284, 29, 43, 42) (4284, 1)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
X, Y = None, None
folds = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(X_train, y_train))
#X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)
#print(X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape)



In [0]:
model_name = "Small_AlexNet_with_Dropout"
def get_model():
  model = Sequential()
  
  model.add(Conv2D(64, kernel_size=5, strides=(1,1), padding='valid', activation='relu', input_shape = (29, 43, 42)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.3))
  model.add(Conv2D(128, kernel_size=3,strides=(1,1), padding='valid', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.3))

  model.add(Conv2D(256, kernel_size=3, strides=(1,1), padding='valid', activation='relu'))
  model.add(Conv2D(512, kernel_size=2, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  
  sgd = optimizers.SGD(lr=0.000001, momentum=0.7, decay=1e-6)
  model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [24]:
filepath= model_name + "-weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model2 = get_model()
for j, (train_idx, val_idx) in enumerate(folds):
  print('\nFold ',j)
  X_train_cv = X_train[train_idx]
  y_train_cv = y_train[train_idx]
  X_valid_cv = X_train[val_idx]
  y_valid_cv= y_train[val_idx]
  # checkpoint
  model2.fit(X_train_cv, y_train_cv, validation_data=(X_valid_cv, y_valid_cv), epochs=10, batch_size=32, callbacks=callbacks_list, shuffle=True)
score = model2.evaluate(X_test,y_test)
print("Test set metrics: %s: %.2f%%" % (model2.metrics_names[1], score[1]*100))
model2_json = model2.to_json()
with open(model_name + ".json", "w") as json_file:
    json_file.write(model2_json)
# serialize weights to HDF5
model2.save_weights(model_name + ".h5")
print("Saved model %s to disk" % (model_name))


Fold  0
Train on 3276 samples, validate on 365 samples
Epoch 1/10
3200/3276 [============================>.] - ETA: 0s - loss: 8.1214 - accuracy: 0.4822
Epoch 00001: accuracy improved from -inf to 0.48291, saving model to Small_AlexNet_with_Dropout-weights-improvement-01.hdf5
3276/3276 [==============================] - 3s 1ms/sample - loss: 8.0845 - accuracy: 0.4829 - val_loss: 2.0249 - val_accuracy: 0.4795
Epoch 2/10
3200/3276 [============================>.] - ETA: 0s - loss: 6.2561 - accuracy: 0.5022
Epoch 00002: accuracy improved from 0.48291 to 0.50214, saving model to Small_AlexNet_with_Dropout-weights-improvement-02.hdf5
3276/3276 [==============================] - 3s 838us/sample - loss: 6.2487 - accuracy: 0.5021 - val_loss: 1.7334 - val_accuracy: 0.4877
Epoch 3/10
3200/3276 [============================>.] - ETA: 0s - loss: 4.8211 - accuracy: 0.5191
Epoch 00003: accuracy improved from 0.50214 to 0.51862, saving model to Small_AlexNet_with_Dropout-weights-improvement-03.hdf5
